In [114]:
%load_ext autoreload
%autoreload
import sys, os 
import copy
from collections import defaultdict, OrderedDict, namedtuple
from itertools import product as cartesian_product
import exrex
import json
import numpy as np
import pandas as pd
import random
import re
import scipy
import scipy.special
from termcolor import colored
import traceback

sys.path.append('..')
from data.model_info import model_info
from process import (
    GPT3, MockGPT3,
    read_cache, 
)
from process_transformers import Runner
import util
from util import set_seed

import torch
import torch.nn as nn
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[Errno 2] No such file or directory: 'api-key'
Falling back to MockGPT3


In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

In [276]:
def add_fields(df):
    df['year'] = list(map(lambda _: _.year, map(eval, df['content'].values)))
    df['month'] = list(map(lambda _: _.month, map(eval, df['content'].values))) # df['content'].apply(eval).month
    df['day'] = list(map(lambda _: _.day, map(eval, df['content'].values)))
    df['train'] = df['prompt'].apply(lambda _: _.split('\n')[::2]).apply(lambda x1: list(map(lambda x2: x2.replace('Input: ', ''), x1))).apply(lambda x3: x3[:-1])
    return df
CSV_PATH = '../results_error_analysis.csv'
# CSV_PATH = '../results_error_analysis_templates.csv'
full_df = load_df(CSV_PATH); print(len(full_df))
full_df = add_fields(full_df)
condition2 = (full_df.engine == 'davinci')
keys2 = ['engine','rel','x','pred','y','num_examples','train']
print(len(full_df[condition2]))
full_df[condition2][keys2]


6563
1563


,engine,rel,x,pred,y,num_examples,train
4000,davinci,EQUALS,2012-09-26,!09!26!2012!,!09!26!2012!,3,"[2014-06-01, 2007-12-13, 2010-09-23]"
4001,davinci,EQUALS,2017-05-26,!05!26!2017!,!05!26!2017!,3,"[2016-08-06, 2005-07-23, 1980-10-04]"
4002,davinci,EQUALS,1995-01-28,!01!28!1995!,!01!28!1995!,3,"[1973-05-18, 1998-06-23, 1979-05-16]"
4003,davinci,EQUALS,1978-06-13,!06!13!1978!,!06!13!1978!,3,"[1981-12-07, 1988-01-11, 2013-09-13]"
4004,davinci,EQUALS,1999-06-03,!06!03!1999!,!06!03!1999!,3,"[2011-10-06, 2017-02-16, 2012-04-25]"
4005,davinci,EQUALS,1989-07-09,!07!09!1989!,!07!09!1989!,3,"[2005-11-16, 1979-09-05, 1974-01-05]"
4006,davinci,EQUALS,1989-09-09,!09!09!1989!,!09!09!1989!,3,"[2004-04-01, 1974-11-16, 1976-01-02]"
4007,davinci,EQUALS,1988-08-17,!08!17!1988!,!08!17!1988!,3,"[1971-10-06, 1991-04-13, 2007-06-14]"
4008,davinci,EQUALS,1979-08-02,!08!02!1979!,!08!02!1979!,3,"[1993-12-06, 1990-08-25, 2003-11-11]"
4009,davinci,EQUALS,1979-10-10,!10!10!1979!,!10!10!1979!,3,"[1975-03-09, 1985-12-13, 2003-12-22]"


In [280]:
len(full_df[full_df.year == 2019])

129

In [278]:
9./129

0.06976744186046512

In [283]:
%load_ext autoreload
%autoreload
from sample import load_df, get_latex_stats, get_latex_plot_code, generate_latex_plots, create_date_schema
DateSchema = create_date_schema()
# CSV_PATH = '../results_systematicity.csv'
# CSV_PATH = '../results_error_analysis.csv'
CSV_PATH = '../results_error_analysis_templates.csv'
df = load_df(CSV_PATH); print(len(df))

df['year'] = list(map(lambda _: _.year, map(eval, df['content'].values)))
df['month'] = list(map(lambda _: _.month, map(eval, df['content'].values))) # df['content'].apply(eval).month
df['day'] = list(map(lambda _: _.day, map(eval, df['content'].values)))
df['train'] = df['prompt'].apply(lambda _: _.split('\n')[::2]).apply(lambda x1: list(map(lambda x2: x2.replace('Input: ', ''), x1))).apply(lambda x3: x3[:-1])

# condition = (df.engine == 'ada')
# condition = (df.engine == 'babbage')
# condition = (df.engine == 'curie')
condition = (df.engine == 'davinci')
# condition &= (df.num_examples == 3)
condition &= (df.num_examples == 5)
# condition &= (df.num_examples == 10)
# condition &= (df.num_examples == 15)
# condition &= (df.rel != 'EQUALS')
# condition &= (df.rel == 'EQUALS')
# condition &= (df['templates'].str.contains(_swap_month_day, na=False))
# condition &= (df['x'].str.contains('2019', na=False)) # 9/73, all _swap_month_day, 9/34
# condition &= (df['x'].str.contains('-13', na=False)) # 6/34 '03-13'
# condition &= ~(df['x'].str.contains('2019', na=False)) 
# condition &= (df['x'].str.contains('-23', na=False)) # 5/34 
# condition &= (df.day <= 12)
# condition &= ~(df.day <= 12)
# condition &= (df.day > df.month)

print(len(df[condition]))
keys = ['engine','rel','x','pred','y','num_examples','train','day','templates'] # 'templates'
df[condition][keys]

# [hypothesis] for template == _swap_month_day AND day in range month, train had a month == test day
#     only true for 5/11

# [hypothesis] specific tokens? day in [6, 9, 13, 14, 16, 23, 28]



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
713
17


,engine,rel,x,pred,y,num_examples,train,day,templates
662,davinci,NOT EQUALS,2019-06-12,!12!06!2019!,!06!12!2019!,5,"[2009-11-25, 2007-11-28, 1997-03-18, 1992-09-24, 1989-04-20]",12,"[[['fp0'], ['day'], ['fp0'], ['month'], ['fp0'], ['year'], ['fp0']]]"
663,davinci,CLOSE,2000-01-02,!00!02!2000!,!01!02!2000!,5,"[1971-06-14, 1972-05-15, 1991-03-09, 1976-03-22, 2000-07-16]",2,"[[['fp0'], ['year_2:4', 'month_minus-one'], ['fp0'], ['month_plus-one', 'day'], ['fp0'], ['year'], ['fp0']]]"
664,davinci,CLOSE,2002-01-09,!02!09!2002!,!01!09!2002!,5,"[1976-04-15, 2000-02-16, 1998-08-15, 2005-01-19, 1970-11-25]",9,"[[['fp0'], ['year_2:4', 'month_plus-one'], ['fp0'], ['day'], ['fp0'], ['year'], ['fp0']]]"
665,davinci,CLOSE,2006-07-16,!06!16!2006!,!07!16!2006!,5,"[2003-07-08, 1972-11-22, 1993-09-01, 2010-10-22, 1979-10-24]",16,"[[['fp0'], ['year_2:4', 'month_minus-one'], ['fp0'], ['day'], ['fp0'], ['year'], ['fp0']]]"
666,davinci,CLOSE,2015-03-13,!13!03!2015!,!03!13!2015!,5,"[1974-05-25, 2001-02-08, 1977-10-05, 2007-06-07, 1996-04-03]",13,"[[['fp0'], ['day'], ['fp0'], ['month'], ['fp0'], ['year'], ['fp0']]]"
667,davinci,NOT EQUALS,2019-08-13,!13!08!2019!,!08!13!2019!,5,"[1994-03-09, 1990-04-21, 1983-10-23, 2006-11-07, 2008-07-03]",13,"[[['fp0'], ['day'], ['fp0'], ['month'], ['fp0'], ['year'], ['fp0']]]"
668,davinci,NOT EQUALS,2005-07-22,!22!07!2005!,!07!22!2005!,5,"[1999-11-15, 1972-12-16, 2014-09-28, 1993-06-06, 2018-07-08]",22,"[[['fp0'], ['day'], ['fp0'], ['month'], ['fp0'], ['year'], ['fp0']]]"
669,davinci,CLOSE,2000-01-23,!00!23!2000!,!01!23!2000!,5,"[1972-07-13, 1981-03-18, 1988-12-19, 1999-10-02, 2005-04-25]",23,"[[['fp0'], ['year_2:4', 'month_minus-one'], ['fp0'], ['day'], ['fp0'], ['year'], ['fp0']]]"
670,davinci,CLOSE,1972-01-06,!06!01!1972!,!01!06!1972!,5,"[2019-05-08, 1972-03-08, 1984-06-02, 1996-11-01, 2017-01-26]",6,"[[['fp0'], ['day'], ['fp0'], ['month'], ['fp0'], ['year'], ['fp0']]]"
671,davinci,NOT EQUALS,2019-08-23,!23!08!2019!,!08!23!2019!,5,"[1996-06-04, 1996-09-14, 2000-08-28, 1992-08-22, 1990-03-16]",23,"[[['fp0'], ['day'], ['fp0'], ['month'], ['fp0'], ['year'], ['fp0']]]"


In [255]:
# davinci, n = 5
print(len(df[condition]))
df[condition]['templates'].value_counts()

17


[[['fp0'], ['day'], ['fp0'], ['month'], ['fp0'], ['year'], ['fp0']]]                                            11
[[['fp0'], ['year_2:4', 'month_minus-one'], ['fp0'], ['day'], ['fp0'], ['year'], ['fp0']]]                      2 
[[['fp0'], ['month_plus-one', 'day'], ['fp0'], ['month_plus-one', 'day'], ['fp0'], ['year'], ['fp0']]]          1 
[[['fp0'], ['month_plus-one', 'day'], ['fp0'], ['month', 'day_minus-one'], ['fp0'], ['year'], ['fp0']]]         1 
[[['fp0'], ['year_2:4', 'month_plus-one'], ['fp0'], ['day'], ['fp0'], ['year'], ['fp0']]]                       1 
[[['fp0'], ['year_2:4', 'month_minus-one'], ['fp0'], ['month_plus-one', 'day'], ['fp0'], ['year'], ['fp0']]]    1 
Name: templates, dtype: int64

In [253]:
# davinci, n = 3
print(len(df[condition]))
df[condition]['templates'].value_counts()

56


[[['fp0'], ['day'], ['fp0'], ['month'], ['fp0'], ['year'], ['fp0']]]                                                                                   23
[[['fp0'], ['year_2:4', 'day'], ['fp0'], ['month'], ['fp0'], ['year'], ['fp0']]]                                                                       8 
[[['fp0'], ['year_2:4', 'month_minus-one'], ['fp0'], ['day'], ['fp0'], ['year'], ['fp0']]]                                                             5 
[[['fp0'], ['year_2:4'], ['fp0'], ['day'], ['fp0'], ['year'], ['fp0']]]                                                                                4 
[[['fp0'], ['month_minus-one', 'day'], ['fp0'], ['month', 'day_plus-one'], ['fp0'], ['year'], ['fp0']]]                                                2 
[[['fp0'], ['day'], ['fp0'], ['day'], ['fp0'], ['year'], ['fp0']]]                                                                                     2 
[[['fp0'], ['year_2:4'], ['fp0'], ['month'], ['fp0'], ['year'], ['fp0']]]   

In [258]:
# curie, n = 15
print(len(df[condition]))
df[condition]['templates'].value_counts()

3


[[['fp0'], ['year_2:4', 'day'], ['fp0'], ['year_2:4', 'day'], ['fp0'], ['year'], ['fp0']]]    2
[[['fp0'], ['day'], ['fp0'], ['day'], ['fp0'], ['year'], ['fp0']]]                            1
Name: templates, dtype: int64

In [266]:
# curie, n = 10
print(len(df[condition]))
df[condition]['templates'].value_counts()

8


[[['fp0'], ['day'], ['fp0'], ['month'], ['fp0'], ['year'], ['fp0']]]                                                              2
[[['fp0'], ['day'], ['fp0'], ['day'], ['fp0'], ['year'], ['fp0']]]                                                                2
[[['fp0'], ['year_2:4', 'month_plus-one', 'day'], ['fp0'], ['year_2:4', 'month_plus-one', 'day'], ['fp0'], ['year'], ['fp0']]]    1
[[['fp0'], ['month_minus-one', 'day'], ['fp0'], ['month', 'day_plus-one'], ['fp0'], ['year'], ['fp0']]]                           1
[[['fp0'], ['month_minus-one', 'day'], ['fp0'], ['month_minus-one', 'day'], ['fp0'], ['year'], ['fp0']]]                          1
[[['fp0'], ['year_2:4'], ['fp0'], ['day'], ['fp0'], ['year'], ['fp0']]]                                                           1
Name: templates, dtype: int64

In [251]:
# curie, n = 5
print(len(df[condition]))
df[condition]['templates'].value_counts()

22


[[['fp0'], ['day'], ['fp0'], ['month'], ['fp0'], ['year'], ['fp0']]]                                       10
[[['fp0'], ['year_2:4'], ['fp0'], ['month'], ['fp0'], ['year'], ['fp0']]]                                  2 
[[['fp0'], ['day'], ['fp0'], ['day'], ['fp0'], ['year'], ['fp0']]]                                         2 
[[['fp0'], ['year_2:4'], ['fp0'], ['day'], ['fp0'], ['year'], ['fp0']]]                                    2 
[[['fp0'], ['2-digit'], ['fp0'], ['day'], ['fp0'], ['year'], ['fp0']]]                                     2 
[[['fp0'], ['year_2:4', 'day'], ['fp0'], ['month'], ['fp0'], ['year'], ['fp0']]]                           1 
[[['fp0'], ['month_plus-one', 'day'], ['fp0'], ['month_plus-one', 'day'], ['fp0'], ['year'], ['fp0']]]     1 
[[['fp0'], ['month_plus-one', 'day'], ['fp0'], ['month', 'day_minus-one'], ['fp0'], ['year'], ['fp0']]]    1 
[[['fp0'], ['month_minus-one', 'day'], ['fp0'], ['month', 'day_plus-one'], ['fp0'], ['year'], ['fp0']]]    1 
Name: temp

In [263]:
# babbage, n = 15
print(len(df[condition]))
df[condition]['templates'].value_counts()

44


[[['fp0'], ['day'], ['fp0'], ['day'], ['fp0'], ['year'], ['fp0']]]                                                                17
[[['fp0'], ['day'], ['fp0'], ['month'], ['fp0'], ['year'], ['fp0']]]                                                              16
[[['fp0'], ['month_plus-one', 'day'], ['fp0'], ['month_plus-one', 'day'], ['fp0'], ['year'], ['fp0']]]                            3 
[[['fp0'], ['year_2:4', 'day'], ['fp0'], ['year_2:4', 'day'], ['fp0'], ['year'], ['fp0']]]                                        2 
[[['fp0'], ['month', 'day'], ['fp0'], ['2-digit'], ['fp0'], ['year'], ['fp0']]]                                                   2 
[[['fp0'], ['month_minus-one', 'day'], ['fp0'], ['month_minus-one', 'day'], ['fp0'], ['year'], ['fp0']]]                          2 
[[['fp0'], ['year_2:4', 'month_plus-one', 'day'], ['fp0'], ['year_2:4', 'month_plus-one', 'day'], ['fp0'], ['year'], ['fp0']]]    1 
[[['fp0'], ['2-digit'], ['fp0'], ['day'], ['fp0'], ['year'], ['fp0']]

In [260]:
# ada, n = 15
print(len(df[condition]))
df[condition]['templates'].value_counts()

563


[[['fp0'], ['day'], ['fp0'], ['day'], ['fp0'], ['year'], ['fp0']]]                                                                383
[[['fp0'], ['month_minus-one', 'day'], ['fp0'], ['month_minus-one', 'day'], ['fp0'], ['year'], ['fp0']]]                          31 
[[['fp0'], ['2-digit'], ['fp0'], ['day'], ['fp0'], ['year'], ['fp0']]]                                                            31 
[[['fp0'], ['day'], ['fp0'], ['month'], ['fp0'], ['year'], ['fp0']]]                                                              30 
[[['fp0'], ['day'], ['fp0'], ['2-digit'], ['fp0'], ['year'], ['fp0']]]                                                            21 
[[['fp0'], ['month_plus-one', 'day'], ['fp0'], ['month_plus-one', 'day'], ['fp0'], ['year'], ['fp0']]]                            18 
[[['fp0'], ['year_2:4', 'day'], ['fp0'], ['year_2:4', 'day'], ['fp0'], ['year'], ['fp0']]]                                        10 
[[['fp0'], ['month_plus-one'], ['fp0'], ['day'], ['fp0'], ['ye

In [238]:
df[(df.engine == 'curie')][keys]

,engine,rel,x,pred,y,num_examples,train,day,test_idx
0,curie,CLOSE,1972-01-06,!06!06!1972!,!01!06!1972!,15,"[1987-01-13, 2005-08-05, 2016-05-21, 2012-02-26, 2004-06-26, 1970-05-24, 2019-04-14, 1994-02-22, 1975-03-14, 1972-09-16, 2019-05-08, 1972-03-08, 1984-06-02, 1996-11-01, 2017-01-26]",6,209
1,curie,CLOSE,2008-05-08,!08!08!2008!,!05!08!2008!,15,"[2001-09-13, 2005-01-24, 2013-07-15, 1972-06-14, 1990-02-19, 1977-01-19, 2000-12-24, 1980-11-06, 2019-05-15, 1993-05-19, 1995-07-28, 1981-09-01, 2004-06-08, 1975-11-07, 2005-10-10]",8,644
2,curie,CLOSE,2004-01-04,!04!04!2004!,!01!04!2004!,15,"[2006-12-14, 1974-01-18, 1988-04-26, 1994-01-13, 1994-04-24, 1997-12-27, 2005-04-05, 2014-02-23, 1996-10-20, 1979-08-01, 2012-07-24, 1993-06-01, 1976-10-23, 2014-07-16, 2015-01-09]",4,996


In [240]:
day_day = "[[['fp0'], ['day'], ['fp0'], ['day'], ['fp0'], ['year'], ['fp0']]]"
df[(df.engine == 'ada') & (df.templates == day_day)][keys]

,engine,rel,x,pred,y,num_examples,train,day
25,ada,CLOSE,1988-08-17,!17!17!1988!,!08!17!1988!,15,"[2011-10-06, 2017-02-16, 2012-04-25, 1999-06-03, 2005-11-16, 1979-09-05, 1974-01-05, 1989-07-09, 2004-04-01, 1974-11-16, 1976-01-02, 1989-09-09, 1971-10-06, 1991-04-13, 2007-06-14]",17
26,ada,CLOSE,2014-05-12,!12!12!2014!,!05!12!2014!,15,"[1970-04-18, 2012-09-22, 2018-01-12, 2007-01-10, 1998-12-14, 1995-06-06, 2017-06-03, 1983-11-08, 1991-05-16, 1985-07-06, 2005-01-24, 2019-08-12, 1983-02-07, 2005-03-03, 1988-09-18]",12
27,ada,CLOSE,1993-11-04,!04!04!1993!,!11!04!1993!,15,"[1974-07-24, 2016-06-09, 1990-08-20, 1973-06-11, 2009-05-11, 1981-10-12, 1984-07-19, 2013-12-24, 1994-04-15, 1992-03-15, 2000-12-10, 2018-04-18, 2003-08-08, 1993-05-21, 1975-07-24]",4
28,ada,CLOSE,1979-03-05,!05!05!1979!,!03!05!1979!,15,"[2006-12-09, 2007-03-11, 1980-04-07, 1988-01-16, 2019-03-11, 2016-01-17, 1970-12-12, 1972-03-03, 2009-01-11, 1972-06-09, 2017-11-11, 2015-03-03, 1984-02-08, 1999-03-24, 2008-11-02]",5
30,ada,CLOSE,1990-05-23,!23!23!1990!,!05!23!1990!,15,"[2010-11-16, 1984-12-20, 1981-03-18, 2003-08-16, 1972-10-17, 1970-03-27, 1975-10-07, 1971-11-22, 1991-10-14, 2005-02-22, 1996-08-05, 2018-07-15, 1996-10-03, 2007-12-14, 2015-06-05]",23
31,ada,CLOSE,2019-04-07,!07!07!2019!,!04!07!2019!,15,"[1970-06-05, 2019-08-13, 1999-04-16, 1995-04-24, 2019-07-12, 1995-10-02, 1995-09-27, 2007-02-06, 2016-05-17, 1970-02-26, 2015-12-18, 2013-03-16, 1988-11-02, 1970-06-20, 1984-12-06]",7
32,ada,CLOSE,2011-10-07,!07!07!2011!,!10!07!2011!,15,"[1985-05-08, 2014-11-24, 1984-08-03, 1995-03-08, 2012-11-09, 2013-08-11, 2006-05-15, 2013-09-18, 1997-07-27, 1986-12-21, 2016-11-01, 1995-12-02, 1997-07-06, 1981-06-22, 1976-10-11]",7
34,ada,CLOSE,2013-12-15,!15!15!2013!,!12!15!2013!,15,"[2013-03-20, 2017-11-23, 1972-12-22, 1993-11-15, 2004-04-28, 2001-06-03, 2002-11-02, 2019-03-09, 2007-10-05, 1992-01-16, 2010-03-06, 1985-08-22, 1982-10-01, 2003-09-03, 1983-06-25]",15
35,ada,CLOSE,1988-02-05,!05!05!1988!,!02!05!1988!,15,"[2004-10-09, 2017-05-15, 2014-07-09, 2017-02-06, 2014-11-17, 2008-08-14, 1996-06-14, 2002-04-03, 1990-04-21, 1976-01-27, 1982-01-14, 1996-11-22, 1987-02-22, 1970-08-05, 2000-04-05]",5
39,ada,CLOSE,2006-11-07,!07!07!2006!,!11!07!2006!,15,"[1997-12-27, 1984-10-12, 1980-02-05, 2012-05-11, 2015-05-10, 2009-04-01, 1994-02-22, 1991-03-24, 1988-12-04, 2008-09-26, 1970-10-27, 1971-11-12, 1993-07-14, 2004-08-13, 2004-08-07]",7


In [264]:
# all
print(len(df))
df['templates'].value_counts()

705


[[['fp0'], ['day'], ['fp0'], ['day'], ['fp0'], ['year'], ['fp0']]]                                                                                     405
[[['fp0'], ['day'], ['fp0'], ['month'], ['fp0'], ['year'], ['fp0']]]                                                                                   90 
[[['fp0'], ['2-digit'], ['fp0'], ['day'], ['fp0'], ['year'], ['fp0']]]                                                                                 34 
[[['fp0'], ['month_minus-one', 'day'], ['fp0'], ['month_minus-one', 'day'], ['fp0'], ['year'], ['fp0']]]                                               33 
[[['fp0'], ['month_plus-one', 'day'], ['fp0'], ['month_plus-one', 'day'], ['fp0'], ['year'], ['fp0']]]                                                 23 
[[['fp0'], ['day'], ['fp0'], ['2-digit'], ['fp0'], ['year'], ['fp0']]]                                                                                 21 
[[['fp0'], ['year_2:4', 'day'], ['fp0'], ['year_2:4', 'day'], ['fp0'],

In [270]:
df['year'].value_counts()

2019    36
2018    31
2009    25
2016    22
2006    22
2015    21
2013    21
2008    21
1984    20
1997    20
1982    18
2004    18
1971    18
2014    17
2005    16
1989    16
2010    15
1985    15
2002    14
1988    14
2011    14
1975    14
1995    13
2017    13
1992    13
2003    13
2000    13
2012    13
1998    12
1987    12
1980    12
2007    12
1981    11
1978    11
1979    11
1986    11
1983    11
1977    10
1973    10
1993    10
1972    10
1990    10
1976    10
1974    8 
1994    8 
2001    7 
1991    6 
1996    6 
1970    6 
1999    3 
Name: year, dtype: int64

In [284]:
# df[df.engine == 'davinci']['year'].value_counts()
full_df[full_df.engine == 'davinci']['year'].value_counts()

1979    52
1995    46
1991    41
2000    40
1998    39
1986    39
1992    38
1993    38
1983    37
1990    37
2016    37
1988    36
2003    36
2019    36
1977    34
2013    33
1981    33
1984    33
2007    33
2001    33
1976    33
1996    32
2005    32
1982    31
2011    31
1972    30
2015    30
1997    29
1974    29
1978    29
1971    29
2006    29
2002    29
2018    29
2004    28
2009    27
2012    27
2017    27
1999    27
2008    26
1987    26
1980    26
1989    25
2014    25
1985    24
2010    23
1975    22
1994    21
1973    21
1970    15
Name: year, dtype: int64

In [272]:
df[df.engine == 'davinci']['year'].value_counts()

2019    9
2015    9
2014    5
2000    5
2006    4
2016    4
2005    4
1972    3
2018    3
2002    3
2009    3
2013    3
2011    2
2017    2
2008    1
1988    1
1975    1
1977    1
1981    1
1986    1
1994    1
1990    1
2007    1
1995    1
1997    1
2001    1
2012    1
1971    1
Name: year, dtype: int64

In [282]:
df[(df.engine == 'davinci') & (df.num_examples == 3)]['year'].value_counts()

2015    7
2019    5
2016    3
2006    3
2000    3
2013    3
2009    3
2018    3
2005    3
2002    2
2014    2
2011    2
1972    2
1994    1
1990    1
1995    1
1997    1
2001    1
1988    1
1981    1
2007    1
2008    1
1977    1
2012    1
2017    1
1971    1
1975    1
1986    1
Name: year, dtype: int64

In [281]:
df[(df.engine == 'davinci') & (df.num_examples == 5)]['year'].value_counts()

2019    4
2014    3
2015    2
2000    2
2006    1
2005    1
1972    1
2002    1
2017    1
2016    1
Name: year, dtype: int64

In [269]:
df[df.year == 2018]['engine'].value_counts()

ada        20
babbage    6 
davinci    3 
curie      2 
Name: engine, dtype: int64

In [229]:
df[condition]['year'].value_counts()

2019    9
2015    7
1972    3
2018    2
2014    2
2005    2
1981    1
2016    1
2013    1
2011    1
2008    1
2002    1
1997    1
1995    1
1990    1
Name: year, dtype: int64

In [228]:
df[condition]['month'].value_counts()

3     6
7     5
1     5
8     4
4     4
5     3
11    2
10    2
6     2
9     1
Name: month, dtype: int64

In [227]:
df[condition]['day'].value_counts()

13    6
23    5
4     2
22    2
16    2
6     2
14    2
9     1
2     1
5     1
7     1
8     1
28    1
12    1
27    1
15    1
18    1
24    1
26    1
1     1
Name: day, dtype: int64

In [195]:
df[condition]['x'].value_counts()

1972-01-06    2
2015-03-13    2
2005-07-22    2
2019-06-12    1
2015-10-04    1
2002-04-14    1
1995-08-23    1
2019-04-13    1
1981-07-23    1
2019-07-28    1
2013-01-23    1
2015-11-01    1
2014-03-13    1
2015-01-14    1
2018-06-04    1
2019-08-23    1
2018-05-08    1
2019-03-13    1
2019-04-26    1
2015-03-18    1
2015-09-16    1
1997-01-16    1
2019-08-13    1
1990-05-23    1
2014-11-24    1
1972-08-05    1
2019-05-15    1
2011-10-07    1
2016-07-27    1
2008-04-02    1
2019-03-09    1
Name: x, dtype: int64

In [187]:
swap_month_day = "[[['fp0'], ['day'], ['fp0'], ['month'], ['fp0'], ['year'], ['fp0']]]"
month_day_close = "[[['fp0'], ['month', 'day_minus-one'], ['fp0'], ['month_plus-one', 'day'], ['fp0'], ['year'], ['fp0']]]"


In [194]:
_swap_month_day = swap_month_day[1:-1].replace('[','\[').replace(']','\]').replace("'","\'")
# df[df['templates'].str.contains("\[\[\'fp0\'\], \[\'day\'\], \[\'fp0\'\], \[\'month\'\], \[\'fp0\'\], \[\'year\'\], \[\'fp0\'\]\]", na=False)][keys]
df[df['templates'].str.contains(_swap_month_day, na=False)][keys]


,engine,rel,x,pred,y,num_examples,train
1,davinci,NOT EQUALS,1990-05-23,!23!05!1990!,!05!23!1990!,3,"[1996-10-03, 2007-12-14, 2015-06-05]"
3,davinci,NOT EQUALS,2011-10-07,!07!10!2011!,!10!07!2011!,3,"[1997-07-06, 1981-06-22, 1976-10-11]"
4,davinci,CLOSE,2019-03-09,!09!03!2019!,!03!09!2019!,3,"[2004-04-28, 2001-06-03, 2002-11-02]"
5,davinci,CLOSE,2019-03-13,!13!03!2019!,!03!13!2019!,3,"[1974-10-03, 2007-04-24, 1989-12-21]"
7,davinci,NOT EQUALS,2015-09-16,!16!09!2015!,!09!16!2015!,3,"[1970-03-04, 2010-04-25, 1997-07-02]"
9,davinci,CLOSE,2015-03-13,!13!03!2015!,!03!13!2015!,3,"[1977-10-05, 2007-06-07, 1996-04-03]"
10,davinci,CLOSE,2018-05-08,!08!05!2018!,!05!08!2018!,3,"[1977-07-10, 1979-06-05, 1984-05-16]"
13,davinci,NOT EQUALS,2005-07-22,!22!07!2005!,!07!22!2005!,3,"[2014-09-28, 1993-06-06, 2018-07-08]"
18,davinci,NOT EQUALS,1995-08-23,!23!08!1995!,!08!23!1995!,3,"[1980-01-28, 1972-09-10, 2018-08-07]"
19,davinci,NOT EQUALS,2019-04-26,!26!04!2019!,!04!26!2019!,3,"[2011-07-09, 2005-10-17, 1975-07-18]"


In [169]:
df[df.templates == '[]'][keys]

,engine,rel,x,pred,y,num_examples,task,templates,train
60,ada,CLOSE,1984-10-18,!08!18!1984!,!10!18!1984!,15,DateSchema,[],"[1988-04-06, 2001-08-09, 1995-06-20, 2000-08-10, 2003-11-18, 2003-09-25, 2005-07-25, 2001-12-28, 2001-10-06, 2000-07-14, 2011-02-22, 1974-01-21, 2010-07-24, 2019-10-08, 1991-06-28]"
68,ada,CLOSE,1993-04-26,!06!26!1993!,!04!26!1993!,15,DateSchema,[],"[2007-01-26, 1984-03-27, 2010-02-28, 1983-02-27, 2000-11-16, 1984-10-15, 2001-08-21, 2005-09-03, 2014-07-10, 1985-06-25, 1975-07-25, 2018-08-04, 1980-11-19, 1979-12-04, 1981-06-17]"
69,ada,CLOSE,1986-08-26,!06!26!1986!,!08!26!1986!,15,DateSchema,[],"[1998-02-23, 1982-06-15, 2007-11-18, 2001-06-05, 1980-01-24, 1988-11-09, 1996-10-17, 1991-01-06, 1997-06-12, 2004-04-24, 2010-03-02, 1988-01-26, 1980-04-11, 2003-10-23, 1970-04-05]"
82,ada,NOT EQUALS,2019-04-22,!22!16!2019!,!04!22!2019!,15,DateSchema,[],"[1991-01-13, 1976-05-10, 1982-10-11, 2000-11-16, 2014-06-01, 1993-11-01, 1979-04-04, 1972-06-27, 1974-10-20, 2011-04-19, 1997-11-25, 1987-11-22, 1979-05-13, 2000-11-25, 1992-06-16]"
90,ada,CLOSE,1987-02-24,!12!24!1987!,!02!24!1987!,15,DateSchema,[],"[1976-10-13, 1986-06-25, 2017-04-07, 1981-03-03, 2005-12-18, 1981-08-26, 1974-03-04, 1970-12-05, 2017-01-14, 2003-06-25, 1985-01-12, 1973-08-03, 2014-01-19, 2008-06-12, 2004-10-26]"
91,ada,NOT EQUALS,2017-01-22,!22!12!2017!,!01!22!2017!,15,DateSchema,[],"[1980-11-17, 1981-10-08, 2012-07-27, 2009-01-20, 1980-12-04, 1976-12-04, 1986-03-28, 1982-12-06, 1971-11-06, 1990-01-24, 2003-01-10, 1982-11-24, 1984-06-16, 2014-10-23, 2000-10-12]"
123,ada,CLOSE,1975-01-05,!05!15!1975!,!01!05!1975!,15,DateSchema,[],"[1970-10-13, 1994-08-26, 1972-09-14, 2014-12-16, 1992-01-07, 1988-03-13, 2002-10-24, 2010-09-24, 2015-08-01, 2016-03-13, 1970-10-26, 1976-01-24, 1983-11-09, 1998-01-17, 1979-10-16]"
125,ada,CLOSE,2018-09-14,!07!14!2018!,!09!14!2018!,15,DateSchema,[],"[1980-07-16, 1997-06-07, 1987-10-01, 1998-01-15, 2012-06-09, 1970-03-03, 1984-06-08, 2001-04-18, 2018-06-03, 1987-10-21, 2009-09-14, 1990-11-16, 2018-09-26, 2000-07-17, 1988-04-07]"
177,ada,NOT EQUALS,2014-04-25,!25!08!2014!,!04!25!2014!,15,DateSchema,[],"[1985-05-22, 1998-03-12, 2001-09-11, 1996-07-09, 1983-06-23, 2015-12-07, 2018-04-27, 2013-08-27, 2002-11-21, 1979-01-17, 1977-06-20, 2015-11-07, 2013-03-18, 2005-03-14, 1976-07-08]"
184,ada,CLOSE,2018-12-26,!02!26!2018!,!12!26!2018!,15,DateSchema,[],"[2012-10-26, 1974-01-15, 1989-06-25, 1987-09-11, 2009-07-03, 1974-12-24, 1970-08-22, 1975-05-10, 2015-03-14, 2014-08-26, 1972-10-05, 1980-06-07, 1989-02-19, 2012-05-28, 1994-02-23]"


In [139]:

swap_month_day = "[[['fp0'], ['day'], ['fp0'], ['month'], ['fp0'], ['year'], ['fp0']]]"
month_day_close = "[[['fp0'], ['month', 'day_minus-one'], ['fp0'], ['month_plus-one', 'day'], ['fp0'], ['year'], ['fp0']]]"
# keys = ['engine','rel','x','pred','y','num_examples','task','templates']
df['train'] = df['prompt'].apply(lambda _: _.split('\n')[::2]).apply(lambda x1: list(map(lambda x2: x2.replace('Input: ', ''), x1))).apply(lambda x3: x3[:-1])
keys = ['engine','rel','x','pred','y','num_examples','task','templates','train']
df[df.templates == swap_month_day][keys]
# df[df.templates == month_day_close][keys]


,engine,rel,x,pred,y,num_examples,task,templates,train
563,davinci,NOT EQUALS,2019-06-12,!12!06!2019!,!06!12!2019!,5,DateSchema,"[[['fp0'], ['day'], ['fp0'], ['month'], ['fp0'], ['year'], ['fp0']]]","[2009-11-25, 2007-11-28, 1997-03-18, 1992-09-24, 1989-04-20]"
567,davinci,CLOSE,2015-03-13,!13!03!2015!,!03!13!2015!,5,DateSchema,"[[['fp0'], ['day'], ['fp0'], ['month'], ['fp0'], ['year'], ['fp0']]]","[1974-05-25, 2001-02-08, 1977-10-05, 2007-06-07, 1996-04-03]"
568,davinci,NOT EQUALS,2019-08-13,!13!08!2019!,!08!13!2019!,5,DateSchema,"[[['fp0'], ['day'], ['fp0'], ['month'], ['fp0'], ['year'], ['fp0']]]","[1994-03-09, 1990-04-21, 1983-10-23, 2006-11-07, 2008-07-03]"
569,davinci,NOT EQUALS,2005-07-22,!22!07!2005!,!07!22!2005!,5,DateSchema,"[[['fp0'], ['day'], ['fp0'], ['month'], ['fp0'], ['year'], ['fp0']]]","[1999-11-15, 1972-12-16, 2014-09-28, 1993-06-06, 2018-07-08]"
571,davinci,CLOSE,1972-01-06,!06!01!1972!,!01!06!1972!,5,DateSchema,"[[['fp0'], ['day'], ['fp0'], ['month'], ['fp0'], ['year'], ['fp0']]]","[2019-05-08, 1972-03-08, 1984-06-02, 1996-11-01, 2017-01-26]"
572,davinci,NOT EQUALS,2019-08-23,!23!08!2019!,!08!23!2019!,5,DateSchema,"[[['fp0'], ['day'], ['fp0'], ['month'], ['fp0'], ['year'], ['fp0']]]","[1996-06-04, 1996-09-14, 2000-08-28, 1992-08-22, 1990-03-16]"
573,davinci,NOT EQUALS,2014-11-24,!24!11!2014!,!11!24!2014!,5,DateSchema,"[[['fp0'], ['day'], ['fp0'], ['month'], ['fp0'], ['year'], ['fp0']]]","[1979-08-02, 2001-11-07, 1991-07-28, 1990-06-03, 1975-06-01]"
576,davinci,CLOSE,2014-03-13,!13!03!2014!,!03!13!2014!,5,DateSchema,"[[['fp0'], ['day'], ['fp0'], ['month'], ['fp0'], ['year'], ['fp0']]]","[2012-09-03, 2010-08-08, 2013-01-08, 2003-07-06, 1993-05-20]"
577,davinci,NOT EQUALS,2015-03-18,!18!03!2015!,!03!18!2015!,5,DateSchema,"[[['fp0'], ['day'], ['fp0'], ['month'], ['fp0'], ['year'], ['fp0']]]","[2013-04-06, 2010-01-21, 1989-05-06, 1970-03-07, 2009-07-01]"
578,davinci,CLOSE,2016-07-27,!27!07!2016!,!07!27!2016!,5,DateSchema,"[[['fp0'], ['day'], ['fp0'], ['month'], ['fp0'], ['year'], ['fp0']]]","[1983-04-02, 1994-01-09, 2011-10-15, 1977-09-16, 2000-02-23]"


In [ ]:
%load_ext autoreload
%autoreload
from sample import load_df, get_latex_stats, get_latex_plot_code, generate_latex_plots
# CSV_PATH = '../results_systematicity.csv'
CSV_PATH = '../results_error_analysis_templates.csv'
df = load_df(CSV_PATH); print(len(df))
# df = df[df.engine == 'ada']
# df = df[df.engine == 'davinci']
# df = df[df.num_examples == 3]
# df = df[df.rel != 'EQUALS']
# df = df[df.task == 'DateSchema']
# df = df[df.task == 'unnatural_addition']
# df = df[df.task != 'arithmetic_in_words']
# df = df[~df['perplexity_natural'].isnull()]
# df = df[df.index < 50]
# df = df[df.num_examples == 50]
# print(len(df))
# condition = (df.engine == 'ada')
condition = (df.engine == 'davinci')
condition &= (df.num_examples == 3)
# condition &= (df.num_examples == 5)
# condition &= (df.rel != 'EQUALS')
print(len(df[condition]))
# df['train'] = df['prompt'].apply(lambda _: _.split('\n')[::2]).apply(lambda x1: list(map(lambda x2: x2.replace('Input: ', ''), x1))).apply(lambda x3: x3[:-1])
df['train'] = df['prompt'].apply(lambda _: _.split('\n')[::2]).apply(lambda x1: list(map(lambda x2: x2.replace('Input: ', ''), x1))).apply(lambda x3: x3[:-1])
keys = ['engine','rel','x','pred','y','num_examples','task','templates','train']
df[condition][keys]
# df[['engine','rel','x','pred','y','num_examples','task','templates']]
# df[['engine','rel','x','pred','y','perplexity_pred','perplexity_y','perplexity_natural','num_examples','task']].sort_values(['task', 'perplexity_y', 'perplexity_pred'])
# df[['engine','rel','x','pred','y','perplexity_pred','perplexity_y','perplexity_natural','num_examples','task']] \
#     .sort_values(['task', 'perplexity_natural', 'perplexity_pred', 'perplexity_y']).reindex()
#     .sort_values(['task', 'perplexity_natural', 'perplexity_y', 'perplexity_pred']).reindex()

#     .sort_values(['task', 'perplexity_y', 'perplexity_pred'])

In [98]:
df.sort_values(['perplexity_y'], ascending=[False])[['x','pred','y']]

,x,pred,y
863,2019-05-15,!15!05!2019!,!05!15!2019!
655,2014-03-13,!13!03!2014!,!03!13!2014!
132,2019-08-13,!13!08!2019!,!08!13!2019!
161,2000-01-23,!00!23!2000!,!01!23!2000!
24,2019-06-12,!12!06!2019!,!06!12!2019!
565,2019-08-23,!23!08!2019!,!08!23!2019!
59,2006-07-16,!06!16!2006!,!07!16!2006!
575,2014-11-24,!24!11!2014!,!11!24!2014!
155,2005-07-22,!22!07!2005!,!07!22!2005!
34,2000-01-02,!00!02!2000!,!01!02!2000!


In [96]:
train_examples, test_examples

(array([['2019-05-15', '!05!15!2019!'],
        ['2014-03-13', '!03!13!2014!'],
        ['2019-08-13', '!08!13!2019!'],
        ['2000-01-23', '!01!23!2000!'],
        ['2019-06-12', '!06!12!2019!']], dtype=object),
 array([['2019-08-23', '!08!23!2019!'],
        ['2006-07-16', '!07!16!2006!'],
        ['2014-11-24', '!11!24!2014!'],
        ['2005-07-22', '!07!22!2005!'],
        ['2000-01-02', '!01!02!2000!'],
        ['2002-01-09', '!01!09!2002!'],
        ['2015-03-13', '!03!13!2015!'],
        ['2017-12-13', '!12!13!2017!'],
        ['2016-07-27', '!07!27!2016!'],
        ['1972-01-06', '!01!06!1972!'],
        ['2015-03-18', '!03!18!2015!'],
        ['2014-11-12', '!11!12!2014!'],
        ['2019-06-18', '!06!18!2019!']], dtype=object))

In [15]:
train_examples = df.sort_values(['perplexity_y'], ascending=[False])[:5][['x', 'y']].values
test_examples = df.sort_values(['perplexity_y'], ascending=[False])[5:][['x', 'y']].values
print(len(test_examples))
cache = read_cache('../cache_GPT3.jsonl')
gpt3 = GPT3(cache)

959


In [106]:
import openai
openai.api_key = 'sk-K6fDvnq5yv42W99Fpxc4moOt3o3zd7oRHmrPu3vX'
engine = 'davinci'
kwargs = {
    'temperature': 0, 
    'prefix': None, 
    'engine': engine, 
    'max_tokens': 20, 
    'staged': False, 
    'return_kwargs': True,
    'stop': '\n',
    'verbose': False,
    'logprobs': 100,
    'echo': True,
}

for x, y in test_examples:
    response = gpt3.few_shot(train_examples, x=x, y=y, **kwargs)
    text = response[0]['choices'][0]['text'].split('\nOutput: ')[-1]
    print(x, text, y, ('EQUALS' if y == text else 'NOT EQUALS'))
#     break
# response

2019-08-23 !08!23!2019! !08!23!2019! EQUALS
2006-07-16 !07!16!2006! !07!16!2006! EQUALS
2014-11-24 !11!24!2014! !11!24!2014! EQUALS
2005-07-22 !07!22!2005! !07!22!2005! EQUALS
2000-01-02 !01!02!2000! !01!02!2000! EQUALS
2002-01-09 !01!09!2002! !01!09!2002! EQUALS
2015-03-13 !03!13!2015! !03!13!2015! EQUALS
2017-12-13 !12!13!2017! !12!13!2017! EQUALS
2016-07-27 !07!27!2016! !07!27!2016! EQUALS
1972-01-06 !01!06!1972! !01!06!1972! EQUALS
2015-03-18 !03!18!2015! !03!18!2015! EQUALS
2014-11-12 !11!12!2014! !11!12!2014! EQUALS
2019-06-18 !06!18!2019! !06!18!2019! EQUALS


In [112]:
import openai
openai.api_key = 'sk-K6fDvnq5yv42W99Fpxc4moOt3o3zd7oRHmrPu3vX'
engine = 'davinci'
kwargs = {
    'temperature': 0, 
    'prefix': None, 
    'engine': engine, 
    'max_tokens': 20, 
    'staged': False, 
    'return_kwargs': True,
    'stop': '\n',
    'verbose': False,
    'logprobs': 100,
#     'echo': True,
}

for x, y in test_examples:
    response = gpt3.few_shot(train_examples, x=x, y=y, **kwargs)
    text = response[0]['choices'][0]['text'].lstrip()
    print(x, text, y, ('EQUALS' if y == text else 'NOT EQUALS'))
#     break
# response

2019-08-23 !08!23!2019! !08!23!2019! EQUALS
2006-07-16 !07!16!2006! !07!16!2006! EQUALS
2014-11-24 !11!24!2014! !11!24!2014! EQUALS
2005-07-22 !07!22!2005! !07!22!2005! EQUALS
2000-01-02 !01!02!2000! !01!02!2000! EQUALS
2002-01-09 !01!09!2002! !01!09!2002! EQUALS
2015-03-13 !03!13!2015! !03!13!2015! EQUALS
2017-12-13 !12!13!2017! !12!13!2017! EQUALS
2016-07-27 !07!27!2016! !07!27!2016! EQUALS
1972-01-06 !01!06!1972! !01!06!1972! EQUALS
2015-03-18 !03!18!2015! !03!18!2015! EQUALS
2014-11-12 !11!12!2014! !11!12!2014! EQUALS
2019-06-18 !06!18!2019! !06!18!2019! EQUALS


In [16]:
df = load_df(CSV_PATH); print(len(df))
df = df[df.engine == 'davinci']
df = df[df.rel == 'EQUALS']
df = df[df.task == 'DateSchema']
df = df[df.num_examples == 5]
train_examples = df[df.rel == 'EQUALS'].sort_values(['perplexity_y'], ascending=[True])[:5][['x', 'y']].values
# test_examples = df.sort_values(['perplexity_y'], ascending=[False])[5:][['x', 'y']].values

1402


In [7]:
cache = read_cache('../cache_GPT3.jsonl')
gpt3 = GPT3(cache)
len(gpt3.cache), gpt3.cache['__filename__']

(10805, '../cache_GPT3.jsonl')

In [53]:
import openai
openai.api_key = 'sk-K6fDvnq5yv42W99Fpxc4moOt3o3zd7oRHmrPu3vX'
engine = 'davinci'
kwargs = {
    'temperature': 0, 
    'prefix': None, 
    'engine': engine, 
    'max_tokens': 20, 
    'staged': False, 
    'return_kwargs': True,
    'stop': '\n',
    'verbose': False,
    'logprobs': 100,
    'echo': True,
}

for x, y in test_examples:
    response = gpt3.few_shot(train_examples, x=x, y=y, **kwargs)
    text = response[0]['choices'][0]['text'].split('\nOutput: ')[-1]
    print(x, text, y, ('EQUALS' if y == text else 'NOT EQUALS'))
#     break
# response

2019-08-23 !23!08!2019! !08!23!2019! NOT EQUALS
2006-07-16 !07!16!2006! !07!16!2006! EQUALS
2014-11-24 !11!24!2014! !11!24!2014! EQUALS
2005-07-22 !07!22!2005! !07!22!2005! EQUALS
2000-01-02 !00!02!2000! !01!02!2000! NOT EQUALS
1976-08-03 !08!03!1976! !08!03!1976! EQUALS


KeyboardInterrupt: 

In [119]:
train_examples

array([['1998-08-12', '!08!12!1998!'],
       ['1999-05-19', '!05!19!1999!'],
       ['1993-09-01', '!09!01!1993!'],
       ['2014-06-19', '!06!19!2014!'],
       ['1976-10-11', '!10!11!1976!']], dtype=object)

In [17]:
train_examples

array([['1978-04-23', '!04!23!1978!'],
       ['1986-08-18', '!08!18!1986!'],
       ['1981-08-08', '!08!08!1981!'],
       ['1989-08-14', '!08!14!1989!'],
       ['1984-03-20', '!03!20!1984!']], dtype=object)

In [130]:
df[df.rel == 'EQUALS'].sort_values(['perplexity_y'], ascending=[True])[:5]

,content_un,tgt_form_un,tgt_form,perplexity_pred,src_form_un,task,perplexity_y,y,test_idx,pred,prompt,x,rel,perplexity_natural,engine,src_form,num_examples,schema_type,content,temperature,max_tokens,stop
1066,False,2.0,4.0,1.000544,0.0,DateSchema,1.000544,!08!12!1998!,0,!08!12!1998!,Input: 1970-04-15\nOutput: !04!15!1970!\nInput: 1977-10-20\nOutput: !10!20!1977!\nInput: 1975-02-26\nOutput: !02!26!1975!\nInput: 1970-06-23\nOutput: !06!23!1970!\nInput: 2016-11-08\nOutput: !11!08!2016!\nInput: 2018-10-07\nOutput: !10!07!2018!\nInput: 2005-08-18\nOutput: !08!18!2005!\nInput: 1976-06-22\nOutput: !06!22!1976!\nInput: 1970-04-16\nOutput: !04!16!1970!\nInput: 1990-05-21\nOutput: !05!21!1990!\nInput: 2006-12-09\nOutput: !12!09!2006!\nInput: 2007-03-11\nOutput: !03!11!2007!\nInput: 1980-04-07\nOutput: !04!07!1980!\nInput: 1988-01-16\nOutput: !01!16!1988!\nInput: 2019-03-11\nOutput: !03!11!2019!\nInput: 2016-01-17\nOutput: !01!17!2016!\nInput: 1970-12-12\nOutput: !12!12!1970!\nInput: 1972-03-03\nOutput: !03!03!1972!\nInput: 2009-01-11\nOutput: !01!11!2009!\nInput: 1972-06-09\nOutput: !06!09!1972!\nInput: 2017-11-11\nOutput: !11!11!2017!\nInput: 2015-03-03\nOutput: !03!03!2015!\nInput: 1984-02-08\nOutput: !02!08!1984!\nInput: 1999-03-24\nOutput: !03!24!1999!\nInput: 2008-11-02\nOutput: !11!02!2008!\nInput: 1979-03-05\nOutput: !03!05!1979!\nInput: 1988-04-06\nOutput: !04!06!1988!\nInput: 2001-08-09\nOutput: !08!09!2001!\nInput: 1995-06-20\nOutput: !06!20!1995!\nInput: 2000-08-10\nOutput: !08!10!2000!\nInput: 2003-11-18\nOutput: !11!18!2003!\nInput: 2003-09-25\nOutput: !09!25!2003!\nInput: 2005-07-25\nOutput: !07!25!2005!\nInput: 2001-12-28\nOutput: !12!28!2001!\nInput: 2001-10-06\nOutput: !10!06!2001!\nInput: 2000-07-14\nOutput: !07!14!2000!\nInput: 2011-02-22\nOutput: !02!22!2011!\nInput: 1974-01-21\nOutput: !01!21!1974!\nInput: 2010-07-24\nOutput: !07!24!2010!\nInput: 2019-10-08\nOutput: !10!08!2019!\nInput: 1991-06-28\nOutput: !06!28!1991!\nInput: 1984-10-18\nOutput: !10!18!1984!\nInput: 2009-11-25\nOutput: !11!25!2009!\nInput: 2007-11-28\nOutput: !11!28!2007!\nInput: 1997-03-18\nOutput: !03!18!1997!\nInput: 1992-09-24\nOutput: !09!24!1992!\nInput: 1989-04-20\nOutput: !04!20!1989!\nInput: 2019-06-12\nOutput: !06!12!2019!\nInput: 2006-11-28\nOutput: !11!28!2006!\nInput: 1994-11-22\nOutput: !11!22!1994!\nInput: 1998-08-12\nOutput:,1998-08-12,EQUALS,4.766079,davinci,0.0,50,DateSchema,"DateSchema(year=0, month=0, day=0)",0,20,\n
1071,False,2.0,4.0,1.000580,0.0,DateSchema,1.000580,!05!19!1999!,0,!05!19!1999!,Input: 2010-10-22\nOutput: !10!22!2010!\nInput: 1979-10-24\nOutput: !10!24!1979!\nInput: 2006-07-16\nOutput: !07!16!2006!\nInput: 1970-03-04\nOutput: !03!04!1970!\nInput: 2010-04-25\nOutput: !04!25!2010!\nInput: 1997-07-02\nOutput: !07!02!1997!\nInput: 2015-09-16\nOutput: !09!16!2015!\nInput: 2016-06-13\nOutput: !06!13!2016!\nInput: 1988-05-02\nOutput: !05!02!1988!\nInput: 2011-11-16\nOutput: !11!16!2011!\nInput: 2016-12-01\nOutput: !12!01!2016!\nInput: 2007-01-26\nOutput: !01!26!2007!\nInput: 1984-03-27\nOutput: !03!27!1984!\nInput: 2010-02-28\nOutput: !02!28!2010!\nInput: 1983-02-27\nOutput: !02!27!1983!\nInput: 2000-11-16\nOutput: !11!16!2000!\nInput: 1984-10-15\nOutput: !10!15!1984!\nInput: 2001-08-21\nOutput: !08!21!2001!\nInput: 2005-09-03\nOutput: !09!03!2005!\nInput: 2014-07-10\nOutput: !07!10!2014!\nInput: 1985-06-25\nOutput: !06!25!1985!\nInput: 1975-07-25\nOutput: !07!25!1975!\nInput: 2018-08-04\nOutput: !08!04!2018!\nInput: 1980-11-19\nOutput: !11!19!1980!\nInput: 1979-12-04\nOutput: !12!04!1979!\nInput: 1981-06-17\nOutput: !06!17!1981!\nInput: 1993-04-26\nOutput: !04!26!1993!\nInput: 2008-04-13\nOutput: !04!13!2008!\nInput: 2015-06-10\nOutput: !06!10!2015!\nInput: 1995-07-02\nOutput: !07!02!1995!\nInput: 2017-09-28\nOutput: !09!28!2017!\nInput: 2016-07-10\nOutput: !07!10!2016!\nInput: 2008-05-02\nOutput: !05!02!2008!\nInput: 2011-09-19\nOutput: !09!19!2011!\nInput: 2012-03-21\nOutput: !03!21!2012!\nInput: 1998-12-26\nOutput:

In [122]:
test_examples

array([['2019-08-23', '!08!23!2019!'],
       ['2006-07-16', '!07!16!2006!'],
       ['2014-11-24', '!11!24!2014!'],
       ['2005-07-22', '!07!22!2005!'],
       ['2000-01-02', '!01!02!2000!'],
       ['2002-01-09', '!01!09!2002!'],
       ['2015-03-13', '!03!13!2015!'],
       ['2017-12-13', '!12!13!2017!'],
       ['2016-07-27', '!07!27!2016!'],
       ['1972-01-06', '!01!06!1972!'],
       ['2015-03-18', '!03!18!2015!'],
       ['2014-11-12', '!11!12!2014!'],
       ['2019-06-18', '!06!18!2019!']], dtype=object)

In [19]:
test_examples

array([['2019-08-23', '!08!23!2019!'],
       ['2006-07-16', '!07!16!2006!'],
       ['2014-11-24', '!11!24!2014!'],
       ...,
       ['1981-08-08', '!08!08!1981!'],
       ['1986-08-18', '!08!18!1986!'],
       ['1978-04-23', '!04!23!1978!']], dtype=object)

In [167]:
import openai
openai.api_key = 'sk-K6fDvnq5yv42W99Fpxc4moOt3o3zd7oRHmrPu3vX'
engine = 'davinci'
kwargs = {
    'temperature': 0, 
#     'prefix': None, 
    'engine': engine, 
    'max_tokens': 20, 
    'staged': False, 
#     'return_kwargs': True,
    'stop': '\n',
    'verbose': False,
    'logprobs': 100,
#     'echo': True,
}

test_df = df[df.rel != 'EQUALS'].sort_values(['perplexity_y'], ascending=[False])[5:-1]
print(len(test_df))
for row in test_df.itertuples():
    print(row.prompt)
#     response = gpt3.few_shot(train_examples, x=x, y=y, **kwargs)
    response = gpt3.complete(prompt=row.prompt, **kwargs)
    text = response['choices'][0]['text'].lstrip()
    print(row.x, text, row.y, ('EQUALS' if row.y == text else 'NOT EQUALS'))
#     break
response_inc = response

11
Input: 1996-06-04
Output: !06!04!1996!
Input: 1996-09-14
Output: !09!14!1996!
Input: 2000-08-28
Output: !08!28!2000!
Input: 1992-08-22
Output: !08!22!1992!
Input: 1990-03-16
Output: !03!16!1990!
Input: 2019-08-23
Output:
2019-08-23 !23!08!2019! !08!23!2019! NOT EQUALS
Input: 2003-07-08
Output: !07!08!2003!
Input: 1972-11-22
Output: !11!22!1972!
Input: 1993-09-01
Output: !09!01!1993!
Input: 2010-10-22
Output: !10!22!2010!
Input: 1979-10-24
Output: !10!24!1979!
Input: 2006-07-16
Output:
2006-07-16 !06!16!2006! !07!16!2006! NOT EQUALS
Input: 1979-08-02
Output: !08!02!1979!
Input: 2001-11-07
Output: !11!07!2001!
Input: 1991-07-28
Output: !07!28!1991!
Input: 1990-06-03
Output: !06!03!1990!
Input: 1975-06-01
Output: !06!01!1975!
Input: 2014-11-24
Output:
2014-11-24 !24!11!2014! !11!24!2014! NOT EQUALS
Input: 1999-11-15
Output: !11!15!1999!
Input: 1972-12-16
Output: !12!16!1972!
Input: 2014-09-28
Output: !09!28!2014!
Input: 1993-06-06
Output: !06!06!1993!
Input: 2018-07-08
Output: !07!08!2

In [215]:
import openai
openai.api_key = 'sk-K6fDvnq5yv42W99Fpxc4moOt3o3zd7oRHmrPu3vX'
engine = 'davinci'
for temperature in [0,.5]:
    kwargs = {
        'temperature': 0, 
    #     'prefix': None, 
        'engine': engine, 
        'max_tokens': 20, 
        'staged': False, 
    #     'return_kwargs': True,
        'stop': '\n',
        'verbose': False,
        'logprobs': 100,
    #     'echo': True,
        'temperature': temperature, # 1-0.379, (4/11) # 0.379, (5/11) # .5, # 1.33, # 1.4142, # 1.25, # 1.5, # 2, # .5, (7/11) # 1., (1/11) # .1, (3/11)
    } # notably, .5 is around the boundary?

    test_df = df[df.rel != 'EQUALS'].sort_values(['perplexity_y'], ascending=[False])[5:-1]
    # print(len(test_df))
    for row in test_df.itertuples():
    #     print(row.prompt)
    #     response = gpt3.few_shot(train_examples, x=x, y=y, **kwargs)
#         print(row.prompt)
#         prompt = '\n'.join(np.array(row.prompt.split('\n'))[[0,1,2,3,4,5,-2,-1]]) # 6/11
#         prompt = '\n'.join(np.array(row.prompt.split('\n'))[[0,1,2,3,-2,-1]])
        prompt = '\n'.join(np.array(row.prompt.split('\n'))[[0,1,-2,-1]])
        print(prompt)
        response = gpt3.complete(prompt=prompt, **kwargs)
        text = response['choices'][0]['text'].lstrip()
        print(row.x, text, row.y, ('EQUALS' if row.y == text else 'NOT EQUALS'))
#         break
    print()
response_inc = response

Input: 1996-06-04
Output: !06!04!1996!
Input: 2019-08-23
Output:
2019-08-23 !08!23!2019! !08!23!2019! EQUALS
Input: 2003-07-08
Output: !07!08!2003!
Input: 2006-07-16
Output:
2006-07-16 !06!07!2006! !07!16!2006! NOT EQUALS
Input: 1979-08-02
Output: !08!02!1979!
Input: 2014-11-24
Output:
2014-11-24 !11!24!2014! !11!24!2014! EQUALS
Input: 1999-11-15
Output: !11!15!1999!
Input: 2005-07-22
Output:
2005-07-22 !7!22!2005! !07!22!2005! NOT EQUALS
Input: 1971-06-14
Output: !06!14!1971!
Input: 2000-01-02
Output:
2000-01-02 !00!02!2000! !01!02!2000! NOT EQUALS
Input: 1976-04-15
Output: !04!15!1976!
Input: 2002-01-09
Output:
2002-01-09 !02!09!2002! !01!09!2002! NOT EQUALS
Input: 1974-05-25
Output: !05!25!1974!
Input: 2015-03-13
Output:
2015-03-13 !03!13!2015! !03!13!2015! EQUALS
Input: 1994-01-23
Output: !01!23!1994!
Input: 2017-12-13
Output:
2017-12-13 !01!23!2017! !12!13!2017! NOT EQUALS
Input: 1983-04-02
Output: !04!02!1983!
Input: 2016-07-27
Output:
2016-07-27 !07!27!2016! !07!27!2016! EQUALS


In [203]:
import openai
openai.api_key = 'sk-K6fDvnq5yv42W99Fpxc4moOt3o3zd7oRHmrPu3vX'
engine = 'davinci'
for temperature in [0, .1, .5, 1, 1.25, 1.33, 1.4142, 1.5]:
    kwargs = {
        'temperature': 0, 
    #     'prefix': None, 
        'engine': engine, 
        'max_tokens': 20, 
        'staged': False, 
    #     'return_kwargs': True,
        'stop': '\n',
        'verbose': False,
        'logprobs': 100,
    #     'echo': True,
        'temperature': temperature, # 1-0.379, (4/11) # 0.379, (5/11) # .5, # 1.33, # 1.4142, # 1.25, # 1.5, # 2, # .5, (7/11) # 1., (1/11) # .1, (3/11)
    } # notably, .5 is around the boundary?

    test_df = df[df.rel != 'EQUALS'].sort_values(['perplexity_y'], ascending=[False])[5:-1]
    # print(len(test_df))
    for row in test_df.itertuples():
    #     print(row.prompt)
    #     response = gpt3.few_shot(train_examples, x=x, y=y, **kwargs)
        response = gpt3.complete(prompt=row.prompt, **kwargs)
        text = response['choices'][0]['text'].lstrip()
        print(row.x, text, row.y, ('EQUALS' if row.y == text else 'NOT EQUALS'))
    #     break
response_inc = response

2019-08-23 !23!08!2019! !08!23!2019! NOT EQUALS
2006-07-16 !06!16!2006! !07!16!2006! NOT EQUALS
2014-11-24 !24!11!2014! !11!24!2014! NOT EQUALS
2005-07-22 !22!07!2005! !07!22!2005! NOT EQUALS
2000-01-02 !00!02!2000! !01!02!2000! NOT EQUALS
2002-01-09 !02!09!2002! !01!09!2002! NOT EQUALS
2015-03-13 !13!03!2015! !03!13!2015! NOT EQUALS
2017-12-13 !13!12!2017! !12!13!2017! NOT EQUALS
2016-07-27 !27!07!2016! !07!27!2016! NOT EQUALS
1972-01-06 !06!01!1972! !01!06!1972! NOT EQUALS
2015-03-18 !18!03!2015! !03!18!2015! NOT EQUALS
2019-08-23 !23!08!2019! !08!23!2019! NOT EQUALS
2006-07-16 !06!16!2006! !07!16!2006! NOT EQUALS
2014-11-24 !24!11!2014! !11!24!2014! NOT EQUALS
2005-07-22 !07!22!2005! !07!22!2005! EQUALS
2000-01-02 !00!02!2000! !01!02!2000! NOT EQUALS
2002-01-09 !02!09!2002! !01!09!2002! NOT EQUALS
2015-03-13 !13!03!2015! !03!13!2015! NOT EQUALS
2017-12-13 !13!12!2017! !12!13!2017! NOT EQUALS
2016-07-27 !27!07!2016! !07!27!2016! NOT EQUALS
1972-01-06 !01!06!1972! !01!06!1972! EQUALS


In [201]:
1-0.379

0.621

In [198]:
np.exp(-0.43), np.exp(-1.21), np.exp(-0.9689628736363637)

(0.6505090947233165, 0.2981972794298874, 0.37947639906347497)

In [197]:
import openai
openai.api_key = 'sk-K6fDvnq5yv42W99Fpxc4moOt3o3zd7oRHmrPu3vX'
engine = 'davinci'
kwargs = {
    'temperature': 0, 
#     'prefix': None, 
    'engine': engine, 
    'max_tokens': 20, 
    'staged': False, 
#     'return_kwargs': True,
    'stop': '\n',
    'verbose': False,
    'logprobs': 100,
#     'echo': True,
}

avg = 0.
def inspect_logprobs(response):
    global avg
    response_choice = response['choices'][0]
    print(colored(' | ', 'yellow').join(response_choice['logprobs']['tokens']))
#     arr = response_choice['logprobs']['top_logprobs']
    arr = [response_choice['logprobs']['top_logprobs'][1]]
    for obj in arr:
        obj = OrderedDict(list(sorted(obj.items(), key=lambda x: -x[1]))[:5])
        avg += list(obj.values())[1]
        print(json.dumps(obj, indent=4))
# inspect_logprobs(response_cor)
# inspect_logprobs(response_inc)

print(len(test_df))
for row in test_df.itertuples():
#     print(row.prompt)
#     response = gpt3.few_shot(train_examples, x=x, y=y, **kwargs)
    response = gpt3.complete(prompt=row.prompt, **kwargs)
    text = response['choices'][0]['text'].lstrip()
#     print(row.x, text, row.y, ('EQUALS' if row.y == text else 'NOT EQUALS'))
    inspect_logprobs(response)
avg /= len(test_df)
print('avg:')
print(avg)

11
 ! | 23 | ! | 08 | ! | 2019 | ! | 

{
    "23": -0.43455824,
    "08": -1.2103989,
    "09": -4.7812757,
    "20": -5.3323016,
    "22": -5.508685
}
 ! | 06 | ! | 16 | ! | 2006 | ! | 

{
    "06": -0.59708464,
    "07": -1.2107321,
    "16": -2.114766,
    "13": -5.5542073,
    "10": -5.7854314
}
 ! | 24 | ! | 11 | ! | 2014 | ! | 

{
    "24": -0.5754669,
    "11": -1.0631111,
    "12": -3.2940335,
    "23": -5.087315,
    "14": -5.183621
}
 ! | 22 | ! | 07 | ! | 2005 | ! | 

{
    "22": -0.97679317,
    "07": -1.0593432,
    "05": -1.8609715,
    "08": -3.0913486,
    "06": -3.3954825
}
 ! | 00 | ! | 02 | ! | 2000 | ! | 

{
    "00": -0.61031514,
    "01": -0.9923609,
    "02": -2.639715,
    "07": -6.1695147,
    "0": -6.7250986
}
 ! | 02 | ! | 09 | ! | 2002 | ! | 

{
    "02": -0.6137743,
    "01": -0.93887424,
    "09": -3.3646324,
    "00": -4.066922,
    "12": -6.27039
}
 ! | 13 | ! | 03 | ! | 2015 | ! | 

{
    "13": -0.5569024,
    "03": -0.9103874,
    "01": -5.3939533,
   

In [158]:
df[df.rel != 'EQUALS'][['engine','rel','x','pred','y','perplexity_pred','perplexity_y','perplexity_natural','num_examples','task']].sort_values(['perplexity_y'], ascending=[False])[5:]

,engine,rel,x,pred,y,perplexity_pred,perplexity_y,perplexity_natural,num_examples,task
565,davinci,NOT EQUALS,2019-08-23,!23!08!2019!,!08!23!2019!,1.078867,1.189602,NaN,5,DateSchema
59,davinci,CLOSE,2006-07-16,!06!16!2006!,!07!16!2006!,1.082649,1.168444,NaN,5,DateSchema
575,davinci,NOT EQUALS,2014-11-24,!24!11!2014!,!11!24!2014!,1.085607,1.154559,NaN,5,DateSchema
155,davinci,NOT EQUALS,2005-07-22,!22!07!2005!,!07!22!2005!,1.179169,1.148213,NaN,5,DateSchema
34,davinci,CLOSE,2000-01-02,!00!02!2000!,!01!02!2000!,1.103852,1.148207,NaN,5,DateSchema
37,davinci,CLOSE,2002-01-09,!02!09!2002!,!01!09!2002!,1.088265,1.133215,NaN,5,DateSchema
81,davinci,CLOSE,2015-03-13,!13!03!2015!,!03!13!2015!,1.083020,1.131522,NaN,5,DateSchema
641,davinci,CLOSE,2017-12-13,!13!12!2017!,!12!13!2017!,1.111413,1.124788,NaN,5,DateSchema
805,davinci,CLOSE,2016-07-27,!27!07!2016!,!07!27!2016!,1.081738,1.122945,NaN,5,DateSchema
550,davinci,CLOSE,1972-01-06,!06!01!1972!,!01!06!1972!,1.131801,1.112000,NaN,5,DateSchema


In [176]:
import openai
openai.api_key = 'sk-K6fDvnq5yv42W99Fpxc4moOt3o3zd7oRHmrPu3vX'
engine = 'ada'
kwargs = {
    'temperature': 0, 
    'prefix': None, 
    'engine': engine, 
    'max_tokens': 20, 
    'staged': False, 
    'return_kwargs': True,
    'stop': '\n',
    'verbose': False,
    'logprobs': 100,
#     'echo': True,
}

test_examples = test_df[['x', 'y']].values
for x, y in test_examples:
    response = gpt3.few_shot(train_examples, x=x, y=y, **kwargs)
    text = response[0]['choices'][0]['text'].lstrip()
#     print(x, text, y, ('EQUALS' if y == text else 'NOT EQUALS'))
    inspect_logprobs(response[0])
#     break
response_cor = response[0]

 ! | 08 | ! | 23 | ! | 2019 | ! | 

{
    "08": -0.35306,
    "07": -2.819378,
    "06": -2.9726124,
    "23": -3.4736137,
    "09": -3.7170544
}
 ! | 06 | ! | 16 | ! | 2006 | ! | 

{
    "06": -1.2884406,
    "07": -1.6966742,
    "16": -1.787815,
    "01": -2.660448,
    "03": -2.819929
}
 ! | 11 | ! | 24 | ! | 2014 | ! | 

{
    "11": -1.0511329,
    "24": -1.3690803,
    "12": -2.0151317,
    "04": -2.351895,
    "01": -3.7906728
}
 ! | 07 | ! | 22 | ! | 2005 | ! | 

{
    "07": -1.1137527,
    "05": -1.5475886,
    "02": -1.8095744,
    "01": -2.4759028,
    "03": -2.9696429
}
 ! | 02 | ! | 02 | ! | 2000 | ! | 

{
    "02": -0.18714584,
    "01": -2.0359197,
    "00": -4.4404035,
    "03": -4.7713,
    "04": -5.731026
}
 ! | 09 | ! | 13 | ! | 2002 | ! | 

{
    "09": -0.5608384,
    "01": -1.6315933,
    "02": -2.6706388,
    "03": -2.9916205,
    "08": -3.888927
}
 ! | 03 | ! | 13 | ! | 2015 | ! | 

{
    "03": -0.5492532,
    "05": -2.1975229,
    "13": -2.5495298,
    "01": -2.

In [163]:
test_examples

array([['2019-08-23', '!08!23!2019!'],
       ['2006-07-16', '!07!16!2006!'],
       ['2014-11-24', '!11!24!2014!'],
       ['2005-07-22', '!07!22!2005!'],
       ['2000-01-02', '!01!02!2000!'],
       ['2002-01-09', '!01!09!2002!'],
       ['2015-03-13', '!03!13!2015!'],
       ['2017-12-13', '!12!13!2017!'],
       ['2016-07-27', '!07!27!2016!'],
       ['1972-01-06', '!01!06!1972!'],
       ['2015-03-18', '!03!18!2015!'],
       ['2014-11-12', '!11!12!2014!'],
       ['2019-06-18', '!06!18!2019!']], dtype=object)

In [220]:
import openai
openai.api_key = 'sk-K6fDvnq5yv42W99Fpxc4moOt3o3zd7oRHmrPu3vX'
engine = 'davinci'
kwargs = {
    'temperature': 0, 
#     'prefix': None, 
    'engine': engine, 
    'max_tokens': 20, 
    'staged': False, 
#     'return_kwargs': True,
    'stop': '\n',
    'verbose': False,
    'logprobs': 100,
    'echo': True,
}


print(len(test_df))
for row in test_df.itertuples():
#     print(row.prompt)
#     response = gpt3.few_shot(train_examples, x=x, y=y, **kwargs)
    response = gpt3.complete(prompt=row.prompt, **kwargs)
    text = response['choices'][0]['text'].lstrip()
#     print(row.x, text, row.y, ('EQUALS' if row.y == text else 'NOT EQUALS'))


11
